In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('all_merge.csv')
df.head()

,Brand,Review,Detail,Rating
0,Blue_Buffalo,My dogs love it - but currently out of stock,Yes and yes! Both my border collie and Chihuah...,4.7
1,Blue_Buffalo,Inconsistent quality,My dog has been eating this dog food for 10 ye...,5.0
2,Blue_Buffalo,I've not had problems with this food,I've had no problems with this brand of BB dog...,3.0
3,Blue_Buffalo,Fast and convenient,"Order came fast. My pups love this brand, much...",5.0
4,Blue_Buffalo,My dog loves it!,My dogs have been eating this a few years now ...,4.0


In [3]:
df.dtypes

Brand      object
Review     object
Detail     object
Rating    float64
dtype: object

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1978 entries, 0 to 1977
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Brand   1978 non-null   object 
 1   Review  1978 non-null   object 
 2   Detail  1978 non-null   object 
 3   Rating  1978 non-null   float64
dtypes: float64(1), object(3)
memory usage: 61.9+ KB


In [5]:
df.Rating.value_counts()

5.0    1318
3.0     237
4.0     227
4.7     113
4.8      36
2.0      25
1.0      22
Name: Rating, dtype: int64

## Negative vs Positive Detection

In [6]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [7]:
# Generating sentiment for all the sentence present in the dataset
emptyline=[]
for row in df['Detail']:
    
    vs=analyzer.polarity_scores(row)
    emptyline.append(vs)
# Creating new dataframe with sentiments
df_sentiments=pd.DataFrame(emptyline)
df_sentiments.head()

,neg,neu,pos,compound
0,0.020,0.832,0.148,0.8585
1,0.076,0.860,0.064,-0.2960
2,0.000,0.887,0.113,0.9469
3,0.000,0.575,0.425,0.8122
4,0.051,0.901,0.048,-0.0516


In [8]:
# Merging the sentiments back to reviews dataframe
df_D = pd.concat([df.reset_index(drop=True), df_sentiments], axis=1)
df_D.head(3)

,Brand,Review,Detail,Rating,neg,neu,pos,compound
0,Blue_Buffalo,My dogs love it - but currently out of stock,Yes and yes! Both my border collie and Chihuah...,4.7,0.020,0.832,0.148,0.8585
1,Blue_Buffalo,Inconsistent quality,My dog has been eating this dog food for 10 ye...,5.0,0.076,0.860,0.064,-0.2960
2,Blue_Buffalo,I've not had problems with this food,I've had no problems with this brand of BB dog...,3.0,0.000,0.887,0.113,0.9469


In [19]:
# Convert scores into positive and negetive sentiments using some threshold
df['Like'] = np.where(df_D['compound'] >= 0 , '1','0')
df['compound'] = df_D['compound']
df.head(5)

,Brand,Review,Detail,Rating,nopuntext,stopwtext,lemmtext,length,Like,compound
0,Blue_Buffalo,my dogs love it - but currently out of stock,yes and yes! both my border collie and chihuah...,4.7,yes and yes both my border collie and chihuahu...,yes yes border collie chihuahua mix love it’s ...,yes yes border collie chihuahua mix love it’s ...,74,1,0.8585
1,Blue_Buffalo,inconsistent quality,my dog has been eating this dog food for 10 ye...,5.0,my dog has been eating this dog food for 10 ye...,dog eating dog food 10 years noticed order ama...,dog eating dog food 10 year noticed order amaz...,91,0,-0.2960
2,Blue_Buffalo,i've not had problems with this food,i've had no problems with this brand of bb dog...,3.0,ive had no problems with this brand of bb dog ...,ive no problems brand bb dog food started ship...,ive no problem brand bb dog food started shipm...,142,1,0.9469
3,Blue_Buffalo,fast and convenient,"order came fast. my pups love this brand, much...",5.0,order came fast my pups love this brand much b...,order came fast pups love brand much better,order came fast pup love brand much better,12,1,0.8122
4,Blue_Buffalo,my dog loves it!,my dogs have been eating this a few years now ...,4.0,my dogs have been eating this a few years now ...,dogs eating years love one dogs pancreatitis p...,dog eating year love one dog pancreatitis pres...,51,0,-0.0516


### Text Process

In [20]:
#Import all necessary libraries
import pandas as pd 
import numpy as np 
import re
import nltk
import spacy
import string
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


In [21]:
from nltk.corpus import stopwords 
from textblob import TextBlob
from textblob import Word
# Lower casing and removing punctuations

df['Detail'] = df['Detail'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['Review'] = df['Review'].apply(lambda x: " ".join(x.lower() for x in x.split()))

### Remove Punctuation

In [22]:
PUNCT = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('','', PUNCT))
df["nopuntext"] = df["Detail"].apply(lambda text: remove_punctuation(text))
df["nopuntext"].head()

0    yes and yes both my border collie and chihuahu...
1    my dog has been eating this dog food for 10 ye...
2    ive had no problems with this brand of bb dog ...
3    order came fast my pups love this brand much b...
4    my dogs have been eating this a few years now ...
Name: nopuntext, dtype: object

In [23]:
#Remove STOPWORDS
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))
STOPW = set(stopwords.words('english'))
to_delete = ["no","nor","aren't",'couldn',"couldn't",'didn',"didn't",'doesn',
            "doesn't",'don',"don't",'hadn',"hadn't",'hasn',"hasn't",'haven',"haven't",'isn',
             "isn't",'mightn',"mightn't",'mustn',"mustn't",'needn',"needn't",'no','not', "shan't",
            'shan',"shan't",'shouldn',"shouldn't","that'll",'wasn',"wasn't",'weren',"weren't","won't",
            'wouldn',"wouldn't"]
STOPW.difference_update(to_delete)
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPW])
df["stopwtext"] = df["nopuntext"].apply(lambda text: remove_stopwords(text))
df["stopwtext"].head(15)

0     yes yes border collie chihuahua mix love it’s ...
1     dog eating dog food 10 years noticed order ama...
2     ive no problems brand bb dog food started ship...
3           order came fast pups love brand much better
4     dogs eating years love one dogs pancreatitis p...
5                     medium size bag able mix dog food
6                             dogs love less waste pick
7                                       dog loves brand
8                                 good brand not pittie
9     dogs eat anything noticed take time food eatin...
10                   keeps pets hair feeling soft shiny
11    love convenience delivered door best dog food ...
12        feed blue buffalo products heeler labpitt mix
13    dog eating last year isnt best overall dry foo...
14    dogs take time adjust new food mixed previous ...
Name: stopwtext, dtype: object

In [24]:
#Lemmatization
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [25]:
df["lemmtext"] = df["stopwtext"].apply(lambda text:lemmatize_words(text))
df["lemmtext"].head(15)

0     yes yes border collie chihuahua mix love it’s ...
1     dog eating dog food 10 year noticed order amaz...
2     ive no problem brand bb dog food started shipm...
3            order came fast pup love brand much better
4     dog eating year love one dog pancreatitis pres...
5                     medium size bag able mix dog food
6                                dog love le waste pick
7                                        dog love brand
8                                 good brand not pittie
9     dog eat anything noticed take time food eating...
10                     keep pet hair feeling soft shiny
11    love convenience delivered door best dog food ...
12         feed blue buffalo product heeler labpitt mix
13    dog eating last year isnt best overall dry foo...
14    dog take time adjust new food mixed previous d...
Name: lemmtext, dtype: object

In [26]:
df['length'] = df['Detail'].apply(
    lambda row: min(len(row.split(" ")), len(row)) if isinstance(row, str) else None
)
df

,Brand,Review,Detail,Rating,nopuntext,stopwtext,lemmtext,length,Like,compound
0,Blue_Buffalo,my dogs love it - but currently out of stock,yes and yes! both my border collie and chihuah...,4.7,yes and yes both my border collie and chihuahu...,yes yes border collie chihuahua mix love it’s ...,yes yes border collie chihuahua mix love it’s ...,74,1,0.8585
1,Blue_Buffalo,inconsistent quality,my dog has been eating this dog food for 10 ye...,5.0,my dog has been eating this dog food for 10 ye...,dog eating dog food 10 years noticed order ama...,dog eating dog food 10 year noticed order amaz...,91,0,-0.2960
2,Blue_Buffalo,i've not had problems with this food,i've had no problems with this brand of bb dog...,3.0,ive had no problems with this brand of bb dog ...,ive no problems brand bb dog food started ship...,ive no problem brand bb dog food started shipm...,142,1,0.9469
3,Blue_Buffalo,fast and convenient,"order came fast. my pups love this brand, much...",5.0,order came fast my pups love this brand much b...,order came fast pups love brand much better,order came fast pup love brand much better,12,1,0.8122
4,Blue_Buffalo,my dog loves it!,my dogs have been eating this a few years now ...,4.0,my dogs have been eating this a few years now ...,dogs eating years love one dogs pancreatitis p...,dog eating year love one dog pancreatitis pres...,51,0,-0.0516
...,...,...,...,...,...,...,...,...,...,...
1973,Purina_One,shiny coat happy cat,my three indoor cats love this dry cat food. o...,5.0,my three indoor cats love this dry cat food on...,three indoor cats love dry cat food one older ...,three indoor cat love dry cat food one older f...,35,1,0.7430
1974,Purina_One,cat food,it's been awhile since we've branched out towa...,5.0,its been awhile since weve branched out toward...,awhile since weve branched towards higher end ...,awhile since weve branched towards higher end ...,72,0,-0.4199
1975,Purina_One,"new food, new friends!","""meow, meow, meow, purr, purr. purr. chomp, ch...",4.0,meow meow meow purr purr purr chomp chomp crun...,meow meow meow purr purr purr chomp chomp crun...,meow meow meow purr purr purr chomp chomp crun...,33,1,0.4522
1976,Purina_One,they seems super exciting,the purina one salmon flavor & the turkey flav...,3.0,the purina one salmon flavor the turkey flavo...,purina one salmon flavor turkey flavor ones ca...,purina one salmon flavor turkey flavor one cat...,35,0,-0.3291


In [27]:
#Retrieve the clean data
clean_review=df[['Detail','lemmtext','Rating','length','Like']]
clean_review.head()

,Detail,lemmtext,Rating,length,Like
0,yes and yes! both my border collie and chihuah...,yes yes border collie chihuahua mix love it’s ...,4.7,74,1
1,my dog has been eating this dog food for 10 ye...,dog eating dog food 10 year noticed order amaz...,5.0,91,0
2,i've had no problems with this brand of bb dog...,ive no problem brand bb dog food started shipm...,3.0,142,1
3,"order came fast. my pups love this brand, much...",order came fast pup love brand much better,5.0,12,1
4,my dogs have been eating this a few years now ...,dog eating year love one dog pancreatitis pres...,4.0,51,0


In [28]:
# Average word length feature

def avg_word_length(x):
    words=x.split()
    words_length=[len(word) for word in words]
    
    if len(words) == 0:
        avg_word_length = 0
    else:
        avg_word_length=sum(words_length)/len(words)
    
    return avg_word_length
    


In [29]:
clean_review['avg_word_len']=clean_review["Detail"].apply(lambda text: avg_word_length(text))
clean_review.head(10)

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Detail,lemmtext,Rating,length,Like,avg_word_len
0,yes and yes! both my border collie and chihuah...,yes yes border collie chihuahua mix love it’s ...,4.7,74,1,4.635135
1,my dog has been eating this dog food for 10 ye...,dog eating dog food 10 year noticed order amaz...,5.0,91,0,4.428571
2,i've had no problems with this brand of bb dog...,ive no problem brand bb dog food started shipm...,3.0,142,1,4.147887
3,"order came fast. my pups love this brand, much...",order came fast pup love brand much better,5.0,12,1,4.416667
4,my dogs have been eating this a few years now ...,dog eating year love one dog pancreatitis pres...,4.0,51,0,3.901961
5,medium size bag but able to mix other dog food,medium size bag able mix dog food,5.0,10,1,3.700000
6,"dogs love it, less waste to pick up",dog love le waste pick,5.0,8,1,3.500000
7,dog loves the brand,dog love brand,5.0,4,1,4.000000
8,good brand but not my pittie,good brand not pittie,5.0,6,1,3.833333
9,my dogs eat anything but i noticed they take t...,dog eat anything noticed take time food eating...,5.0,34,1,3.705882


In [30]:
#Special characters
def exclamationmark_count(x):
    words=x.split()
    exmark=[word for word in words if word.startswith('!')]
    exmark_count=[len(exmark[i].split('!'))-1 for i in range(0,len(exmark))]
    return sum(exmark_count)
clean_review['exclamationmark_count']=clean_review["Detail"].apply(lambda text: exclamationmark_count(text))
clean_review.head(10)

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,Detail,lemmtext,Rating,length,Like,avg_word_len,exclamationmark_count
0,yes and yes! both my border collie and chihuah...,yes yes border collie chihuahua mix love it’s ...,4.7,74,1,4.635135,0
1,my dog has been eating this dog food for 10 ye...,dog eating dog food 10 year noticed order amaz...,5.0,91,0,4.428571,0
2,i've had no problems with this brand of bb dog...,ive no problem brand bb dog food started shipm...,3.0,142,1,4.147887,0
3,"order came fast. my pups love this brand, much...",order came fast pup love brand much better,5.0,12,1,4.416667,0
4,my dogs have been eating this a few years now ...,dog eating year love one dog pancreatitis pres...,4.0,51,0,3.901961,0
5,medium size bag but able to mix other dog food,medium size bag able mix dog food,5.0,10,1,3.700000,0
6,"dogs love it, less waste to pick up",dog love le waste pick,5.0,8,1,3.500000,0
7,dog loves the brand,dog love brand,5.0,4,1,4.000000,0
8,good brand but not my pittie,good brand not pittie,5.0,6,1,3.833333,0
9,my dogs eat anything but i noticed they take t...,dog eat anything noticed take time food eating...,5.0,34,1,3.705882,0


### NGrams model prepare for machine learning model

In [31]:
from sklearn.model_selection import train_test_split
#Split the train test data
X=clean_review.drop(['Like','Detail'],axis=1)
y=clean_review[['Like']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42,stratify=y)

In [32]:
#Create ngram modelling
from sklearn.feature_extraction.text import CountVectorizer
vect=CountVectorizer(ngram_range=(1,2),max_features=1000)
X_train_txt=vect.fit_transform(X_train['lemmtext'])
X_train_txt=pd.DataFrame(X_train_txt.toarray(),columns=vect.get_feature_names())
X_train_txt

,10,10 year,100,11,12,14,15,16,16 lb,20,...,wouldn,wouldnt,wrong,year,year ago,year never,year old,yes,yet,yr
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1379,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1380,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1381,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1382,0,0,0,0,0,3,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
X_test_txt=vect.transform(X_test['lemmtext'])
X_test_txt=pd.DataFrame(X_test_txt.toarray(),columns=vect.get_feature_names())
X_test_txt.head()

,10,10 year,100,11,12,14,15,16,16 lb,20,...,wouldn,wouldnt,wrong,year,year ago,year never,year old,yes,yet,yr
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
X_train.index

Int64Index([ 932,   24, 1471,  232,  238, 1070, 1817,   77, 1797,   85,
            ...
            1866,  927, 1796,   63, 1825,  279, 1138, 1354, 1193,  297],
           dtype='int64', length=1384)

In [35]:
#Combine with other features
X_train.index=range(0,1384)
X_train_final=pd.concat([X_train_txt,X_train[['avg_word_len','length','exclamationmark_count']]],axis=1)
X_train_final.head()

,10,10 year,100,11,12,14,15,16,16 lb,20,...,year,year ago,year never,year old,yes,yet,yr,avg_word_len,length,exclamationmark_count
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3.700000,20,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4.090909,11,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4.076923,13,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3.500000,2,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4.062500,32,0


In [36]:
X_test.index

Int64Index([1065, 1234, 1628, 1445,  875,  306, 1976,  146, 1729,  508,
            ...
              29,    2,  842,  544,  564,   10, 1246, 1067,  902,  903],
           dtype='int64', length=594)

In [37]:
X_test.index=range(0,594)
X_test_final=pd.concat([X_test_txt,X_test[['avg_word_len','length','exclamationmark_count']]],axis=1)
X_test_final.head()

,10,10 year,100,11,12,14,15,16,16 lb,20,...,year,year ago,year never,year old,yes,yet,yr,avg_word_len,length,exclamationmark_count
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4.190476,21,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4.333333,33,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4.827586,29,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3.870370,54,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4.142857,7,0


In [38]:
X_train_final=X_train_final.values
X_test_final=X_test_final.values
y_train=y_train.values
y_test=y_test.values

### Fitting Random Forest

In [39]:
from sklearn.metrics import accuracy_score,make_scorer
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier as rbf
from sklearn.exceptions import NotFittedError

In [40]:
pipe_rf = make_pipeline(rbf())

In [41]:
rf_param_grid = {

    'randomforestclassifier__criterion':['gini','entropy'],
    'randomforestclassifier__max_features': ['auto','sqrt','log2',None],
    'randomforestclassifier__max_depth': np.arange(1,50,1),
    'randomforestclassifier__n_estimators': np.arange(1,200,5),
    'randomforestclassifier__min_weight_fraction_leaf': np.arange(0,1,0.05),
    'randomforestclassifier__min_samples_split':np.arange(1,50,1)
    
}

# Perform RandomizedSearchCV
try :
    randomized_search_rf = RandomizedSearchCV(estimator=pipe_rf,param_distributions=rf_param_grid,n_iter=100,scoring=make_scorer(accuracy_score),verbose=1,cv=4)

except NotFittedError as e:
    print(repr(e))

In [42]:
randomized_search_rf.fit(X_train_final,y_train)

Fitting 4 folds for each of 100 candidates, totalling 400 fits


C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\pipeline.py:346: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\pipeline.py:346: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\pipeline.py:346: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\pipeline.py:346: DataConversionWarning: A column-vector y was passed when a 1d 

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\pipeline.py:346: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\ensemble\_forest.py", line 393, in fit
    for i, t in e

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\ensemble\_forest.py", line 393, in fit
    for i, t in enumerate(trees))
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\joblib\parallel.py", line 861, in dispatch_one_batch
    self._dispatch

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\pipeline.py:346: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\pipeline.py:346: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\pipeline.py:346: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\pipeline.py:346: DataConversionWarning: A column-vector y was passed when a 1d 

...

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\pipeline.py:346: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\pipeline.py:346: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\pipeline.py:346: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\pipeline.py:346: DataConversionWarning: A column-vector y was passed when a 1d 

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\pipeline.py:346: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\pipeline.py:346: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\pipeline.py:346: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\pipeline.py:346: DataConversionWarning: A column-vector y was passed when a 1d 

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\ensemble\_forest.py", line 393, in fit
    for i, t in enumerate(trees))
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\joblib\parallel.py", line 861, in dispatch_one_batch
    self._dispatch

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\ensemble\_forest.py", line 393, in fit
    for i, t in enumerate(trees))
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\joblib\parallel.py", line 861, in dispatch_one_batch
    self._dispatch

RandomizedSearchCV(cv=4,
                   estimator=Pipeline(steps=[('randomforestclassifier',
                                              RandomForestClassifier())]),
                   n_iter=100,
                   param_distributions={'randomforestclassifier__criterion': ['gini',
                                                                              'entropy'],
                                        'randomforestclassifier__max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42...
                                        'randomforestclassifier__min_weight_fraction_leaf': array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95]),
                                        'randomforestclassifier__n_estimators': array([  1,   6,  11,  16,  21,  26,  31,  36

In [43]:
print(randomized_search_rf.best_params_)
print(randomized_search_rf.best_score_)

{'randomforestclassifier__n_estimators': 161, 'randomforestclassifier__min_weight_fraction_leaf': 0.0, 'randomforestclassifier__min_samples_split': 9, 'randomforestclassifier__max_features': None, 'randomforestclassifier__max_depth': 17, 'randomforestclassifier__criterion': 'entropy'}
0.8887283236994219


In [62]:
rf_pred = randomized_search_rf.predict(X_test_final)
# Making the Confusion Matrix
cm = confusion_matrix(y_test, rf_pred)
print ("Confusion Matrix:\n",cm)

# Accuracy, Precision and Recall
score1 = accuracy_score(y_test,rf_pred)
score2 = precision_score(y_test,rf_pred, pos_label='0')
score3= recall_score(y_test,rf_pred, pos_label='0')
print("\n")
print("Accuracy is ",round(score1*100,2),"%")
print("Precision is ",round(score2,2))
print("Recall is ",round(score3,2))

Confusion Matrix:
 [[  8  60]
 [  1 525]]


Accuracy is  89.73 %
Precision is  0.89
Recall is  0.12


### Fitting Logistic Regression

In [63]:
from sklearn.linear_model import LogisticRegression

In [64]:
pipe_logis=make_pipeline(LogisticRegression())
pipe_logis

Pipeline(steps=[('logisticregression', LogisticRegression())])

In [65]:
logis_param_grid = {
    'logisticregression__l1_ratio':np.arange(0,1,0.05),
    'logisticregression__C':np.arange(0,10,0.5)}

# Perform RandomizedSearchCV
randomized_search_logis = RandomizedSearchCV(estimator=pipe_logis,param_distributions=logis_param_grid,n_iter=100,scoring=make_scorer(accuracy_score),verbose=1,cv=4)


In [66]:
randomized_search_logis.fit(X_train_final,y_train)

Fitting 4 folds for each of 100 candidates, totalling 400 fits


C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\linear_model\_logistic.py:1319: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SO

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\linear_model\_logistic.py:1319: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  retu

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\linear_model\_logistic.py:1319: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  retu

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\linear_model\_logistic.py:1319: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  retu

...

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\linear_model\_logistic.py:1319: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  retu

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\linear_model\_logistic.py:1319: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\linear_model\_logistic.py:1319: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  retu

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


RandomizedSearchCV(cv=4,
                   estimator=Pipeline(steps=[('logisticregression',
                                              LogisticRegression())]),
                   n_iter=100,
                   param_distributions={'logisticregression__C': array([0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. ,
       6.5, 7. , 7.5, 8. , 8.5, 9. , 9.5]),
                                        'logisticregression__l1_ratio': array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95])},
                   scoring=make_scorer(accuracy_score), verbose=1)

In [67]:
print(randomized_search_logis.best_params_)
print(randomized_search_logis.best_score_)

{'logisticregression__l1_ratio': 0.30000000000000004, 'logisticregression__C': 3.5}
0.8807803468208093


In [69]:
logis_pred=randomized_search_logis.predict(X_test_final)
# Making the Confusion Matrix
cm = confusion_matrix(y_test, logis_pred)
print ("Confusion Matrix:\n",cm)

# Accuracy, Precision and Recall
score1 = accuracy_score(y_test,logis_pred)
score2 = precision_score(y_test,logis_pred,pos_label='0')
score3= recall_score(y_test,logis_pred,pos_label='0')
print("\n")
print("Accuracy is ",round(score1*100,2),"%")
print("Precision is ",round(score2,2))
print("Recall is ",round(score3,2))

Confusion Matrix:
 [[ 19  49]
 [ 28 498]]


Accuracy is  87.04 %
Precision is  0.4
Recall is  0.28


### Naive Bayes

In [70]:
from sklearn.naive_bayes import GaussianNB

In [71]:
gnb = GaussianNB()
gnb.fit(X_train_final,y_train)

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


GaussianNB()

In [72]:
gnb_pred=gnb.predict(X_test_final)
# Making the Confusion Matrix
cm = confusion_matrix(y_test, gnb_pred)
print ("Confusion Matrix:\n",cm)

# Accuracy, Precision and Recall
score1 = accuracy_score(y_test,gnb_pred)
score2 = precision_score(y_test,gnb_pred,pos_label='0')
score3= recall_score(y_test,gnb_pred,pos_label='0')
print("\n")
print("Accuracy is ",round(score1*100,2),"%")
print("Precision is ",round(score2,2))
print("Recall is ",round(score3,2))

Confusion Matrix:
 [[ 35  33]
 [257 269]]


Accuracy is  51.18 %
Precision is  0.12
Recall is  0.51


### Gradient Boosting

In [73]:
from sklearn.ensemble import GradientBoostingClassifier as gbr
pipe_gbr=make_pipeline(gbr())
pipe_gbr

Pipeline(steps=[('gradientboostingclassifier', GradientBoostingClassifier())])

In [74]:
gbr_param_grid = {
    'gradientboostingclassifier__learning_rate':np.arange(0.1,2,0.1),
    'gradientboostingclassifier__max_features': ['auto','sqrt','log2',None],
    'gradientboostingclassifier__max_depth': np.arange(1,50,1),
    'gradientboostingclassifier__n_estimators': np.arange(1,200,5),
    'gradientboostingclassifier__min_weight_fraction_leaf': np.arange(0,1,0.05),
    'gradientboostingclassifier__min_samples_split':np.arange(1,50,1)
    
}

# Perform RandomizedSearchCV
randomized_search_gbr = RandomizedSearchCV(estimator=pipe_gbr,param_distributions=gbr_param_grid,n_iter=100,scoring=make_scorer(accuracy_score),verbose=1,cv=4)


In [75]:
randomized_search_gbr.fit(X_train_final, y_train)

Fitting 4 folds for each of 100 candidates, totalling 400 fits


C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples

...

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Lenovo\.conda\envs\

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\ensemble\_gb.py", line 506, in fit
    sample_weight_val, begin_at_stage, monit

RandomizedSearchCV(cv=4,
                   estimator=Pipeline(steps=[('gradientboostingclassifier',
                                              GradientBoostingClassifier())]),
                   n_iter=100,
                   param_distributions={'gradientboostingclassifier__learning_rate': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9]),
                                        'gradientboostingclassifier__max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 1...
                                        'gradientboostingclassifier__min_weight_fraction_leaf': array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95]),
                                        'gradientboostingclassifier__n_estimators': array([  1,   6,  11,  16,  21,  26,  31,  36,  41,  46,  51,  56,  61,
        66,  71,  76,  81,  86,  91,  96, 101, 106, 111, 116

In [76]:
print(randomized_search_gbr.best_params_)
print(randomized_search_gbr.best_score_)

{'gradientboostingclassifier__n_estimators': 81, 'gradientboostingclassifier__min_weight_fraction_leaf': 0.05, 'gradientboostingclassifier__min_samples_split': 36, 'gradientboostingclassifier__max_features': 'log2', 'gradientboostingclassifier__max_depth': 13, 'gradientboostingclassifier__learning_rate': 0.30000000000000004}
0.8872832369942196


In [77]:
gbr_pred=randomized_search_gbr.predict(X_test_final)
# Making the Confusion Matrix
cm = confusion_matrix(y_test, gbr_pred)
print ("Confusion Matrix:\n",cm)

# Accuracy, Precision and Recall
score1 = accuracy_score(y_test,gbr_pred)
score2 = precision_score(y_test,gbr_pred,pos_label='0')
score3= recall_score(y_test,gbr_pred,pos_label='0')
print("\n")
print("Accuracy is ",round(score1*100,2),"%")
print("Precision is ",round(score2,2))
print("Recall is ",round(score3,2))

Confusion Matrix:
 [[  0  68]
 [  0 526]]


Accuracy is  88.55 %
Precision is  0.0
Recall is  0.0


C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Voting Classifer

In [78]:
from sklearn.ensemble import VotingClassifier
clf_voting = VotingClassifier(voting='soft',estimators=[('gbr', randomized_search_gbr),('randomforest',randomized_search_rf),
                                                        ('logistic',randomized_search_logis),('naive bayes',gnb)])
clf_voting.fit(X_train_final, y_train)

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Fitting 4 folds for each of 100 candidates, totalling 400 fits


C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\ensemble\_gb.py", line 506, in fit
    sample_weight_val, begin_at_stage, monitor)
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\ensemble\_gb.py", line 563, in _fit_stages
    random_state, X_csc, X_csr)
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\ensemble\_gb.py", line 215, in _fit_stage
    check_inpu

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\ensemble\_gb.py", line 506, in fit
    sample_weight_val, begin_at_stage, monitor)
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\ensemble\_gb.py", line 563, in _fit_stages
    random_state, X_csc, X_csr)
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\ensemble\_gb.py", line 215, in _fit_stage
    check_inpu

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\model_selection\_validation.py:619: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\ensemble\_gb.py", line 506, in fit
    sample_weight_val, begin_at_stage, monitor)
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\ensemble\_gb.py", line 563, in _fit_stages
    random_state, X_csc, X_csr)
  File "C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\ensemble\_gb.py", line 215, in _fit_stage
    check_inpu

...

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\linear_model\_logistic.py:1319: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scal

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\linear_model\_logistic.py:1319: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scal

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\linear_model\_logistic.py:1319: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scal

C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\linear_model\_logistic.py:1319: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty))
C:\Users\Lenovo\.conda\envs\oc\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scal

VotingClassifier(estimators=[('gbr',
                              RandomizedSearchCV(cv=4,
                                                 estimator=Pipeline(steps=[('gradientboostingclassifier',
                                                                            GradientBoostingClassifier())]),
                                                 n_iter=100,
                                                 param_distributions={'gradientboostingclassifier__learning_rate': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9]),
                                                                      'gradientboostingclassifier__max_depth': ar...
                                                 n_iter=100,
                                                 param_distributions={'logisticregression__C': array([0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. ,
       6.5, 7. , 7.5, 8. , 8.5, 9. , 9.5]),
                 

In [83]:
voting_pred=clf_voting.predict(X_test_final)
# Making the Confusion Matrix
cm = confusion_matrix(y_test, voting_pred)
print ("Confusion Matrix:\n",cm)

# Accuracy, Precision and Recall
score1 = accuracy_score(y_test,voting_pred)
score2 = precision_score(y_test,voting_pred,pos_label='0')
score3= recall_score(y_test,voting_pred,pos_label='0')
print("\n")
print("Accuracy is ",round(score1*100,2),"%")
print("Precision is ",round(score2,2))
print("Recall is ",round(score3,2))


Confusion Matrix:
 [[  7  61]
 [ 12 514]]


Accuracy is  87.71 %
Precision is  0.37
Recall is  0.1


In [84]:
import joblib
filename = 'sentiment_model.sav'
joblib.dump(clf_voting, filename)

['sentiment_model.sav']

In [85]:
# load the model from disk
loaded_model = joblib.load(filename)
result = loaded_model.score(X_test_final, y_test)

In [82]:
print(result)

0.877104377104377
